In [1]:
from random import sample
import random
from tqdm import tqdm
import my_utils as mu
import pandas as pd
import requests
import time
import DHB as dhb
tqdm.pandas()

In [5]:
mu.riot_api_key

'RGAPI-86d239db-bfc9-4f08-acd4-f110b451241a'

In [6]:
url = 'https://asia.api.riotgames.com/lol/match/v5/matches/KR_5624588631?api_key='+mu.riot_api_key
res = requests.get(url).json()
res

{'metadata': {'dataVersion': '2',
  'matchId': 'KR_5624588631',
  'participants': ['yMZGpvG7VtT9aHrZbk0H5jtpBNw5nphBLje_gAL64GXX9zeqS2vEhzsgXTB_buHa2iK52wfuj7G0LQ',
   'CKSfvilVqlgYKj0jiC4SS3pcQxPYkV_f862DRcIz0QeixmxkUE_hMWpiTtfWoeWq6VDzSKN2bylLfw',
   '_bAnR37cNxyo0bO10JQZ4O_Bi0bxvbt9hn3TgZagIyTl2JLWxYNECWPs_e-DDvWY605IubK4JBo5cg',
   '-rBTdO4Bs43Tjp9FpXdI8Zw8013Fxe8pCnOiyQby4T8U4kgmOxpsV08CD6UFG25fm3Q6y65cAyYokQ',
   '9bUnuQx48amw3rEi3ORj2fl2sYnnQ8iHfUnh3wDlPhroJwY-m4EWVn6jP2clNg-69POC-3BFwLY1vw',
   'cm0ACoCF2ewI4h5PNLNJiud2h96kM2IkgcPSknrQvrrrX_WnHIa5oG8D4vWY_rC_sqfo9WtJ9opzsg',
   'Fn2ECq2hR0QHAnUXGIHi5v00GkgSePRRmn0jW60d6kMbCZayBmSTyXMa2MF9yuG9YTYKhTfA8pCVZg',
   'oTLmL99Mf-G4q7s_abKPbLhKHsWbGPJrplfPB2L0e632JIBdXxLwNCKfL6Q9X-mCRF8FUoGnY1O9cw',
   'ep1ZhHnMTve54NNpNyl38noXNcOmqMAGDIObYqPT3owOM-8OrigjzwRXPcXLdogTyhnuS13s8Qdk_A',
   'cv5gpXo6C1OkZ-0n30KV2jAvYWK6raRzvnaHLYkzB6rG_-UochSMBFnk2kvWtnXmkDcw1JQaSO_Srw']},
 'info': {'gameCreation': 1639412845000,
  'gameDuration': 623,
  'g

In [32]:
df = pd.read_csv("sample.csv")

In [2]:
dhb.db_open()
df = dhb.sql("select * from match_aram")
dhb.db_close()

OPEN
close


In [3]:
df.columns

Index(['Unnamed: 0', 'GAMEID', 'PARTICIPANTID', 'CHAMPIONID', 'CHAMPIONNAME',
       'SPELL1', 'SPELL2', 'MAIN_RUNE', 'SUB_RUNE', 'STAT_RUNE', 'TEAMID',
       'WIN', 'KILLS', 'DEATHS', 'ASSISTS', 'TOTALHEAL', 'TOTALDAMAGETAKEN',
       'TOTALDAMAGEDEALTTOCHAMPIONS', 'PHYSICALDAMAGEDEALTTOCHAMPIONS',
       'MAGICDAMAGEDEALTTOCHAMPIONS', 'TOTALDAMAGESHIELDEDONTEAMMATES',
       'TOTALHEALSONTEAMMATES', 'SELFMITIGATEDDAMAGE', 'TOTALTIMECCDEALT',
       'TIMECCINGOTHERS', 'LARGESTKILLINGSPREE', 'GOLDEARNED', 'STARTING_ITEM',
       'MAIN_ITEM_BUILD', 'SUB_ITEM_BUILD', 'BOOTS', 'SKILL_BUILD'],
      dtype='object')

# . 팀 조합

In [33]:
df["WIN"] = df.apply(lambda x: 1 if x["WIN"] == True else 0 , axis=1)

In [120]:
# 팀 조합승률 구하는 함수
games = df[['CHAMPIONNAME','WIN']].groupby(["CHAMPIONNAME"]).count().rename(columns={"WIN":"GAMES"})
games_win = df[['CHAMPIONNAME','WIN']].groupby(["CHAMPIONNAME"]).sum()
win_rate = pd.merge(games, games_win, on=["CHAMPIONNAME"])
win_rate["rate"] = round(win_rate.WIN / win_rate.GAMES * 100,1)

blue_df = df[df.TEAMID == 100]
red_df = df[df.TEAMID == 200]

blue_test = pd.merge(blue_df,blue_df[["GAMEID","CHAMPIONNAME"]], on=["GAMEID"])
blue_test = blue_test.rename(columns={'CHAMPIONNAME_x':"CHAMPIONNAME","CHAMPIONNAME_y":"TEAMCHAMPION"})
blue_test2 = blue_test[["CHAMPIONNAME",'TEAMCHAMPION',"WIN"]].groupby(["CHAMPIONNAME",'TEAMCHAMPION']).count().rename(columns={"WIN":"games"})
blue_test2 = blue_test2.reset_index()
blue_test2 = blue_test2[blue_test2.CHAMPIONNAME != blue_test2.TEAMCHAMPION]
blue_test2_win = blue_test[["CHAMPIONNAME",'TEAMCHAMPION',"WIN"]].groupby(["CHAMPIONNAME",'TEAMCHAMPION']).sum().reset_index()
blue_test2_win = blue_test2_win[blue_test2_win.CHAMPIONNAME != blue_test2_win.TEAMCHAMPION]
blue_champ_combination = pd.merge(blue_test2,blue_test2_win, on=["CHAMPIONNAME",'TEAMCHAMPION'])

red_test = pd.merge(red_df,red_df[["GAMEID","CHAMPIONNAME"]], on=["GAMEID"])
red_test = red_test.rename(columns={'CHAMPIONNAME_x':"CHAMPIONNAME","CHAMPIONNAME_y":"TEAMCHAMPION"})
red_test2 = red_test[["CHAMPIONNAME",'TEAMCHAMPION',"WIN"]].groupby(["CHAMPIONNAME",'TEAMCHAMPION']).count().rename(columns={"WIN":"games"})
red_test2 = red_test2.reset_index()
red_test2 = red_test2[red_test2.CHAMPIONNAME != red_test2.TEAMCHAMPION]
red_test2_win = red_test[["CHAMPIONNAME",'TEAMCHAMPION',"WIN"]].groupby(["CHAMPIONNAME",'TEAMCHAMPION']).sum().reset_index()
red_test2_win = red_test2_win[red_test2_win.CHAMPIONNAME != red_test2_win.TEAMCHAMPION]
red_champ_combination = pd.merge(red_test2,red_test2_win, on=["CHAMPIONNAME",'TEAMCHAMPION'])

champ_combination = blue_champ_combination.append(red_champ_combination).groupby(["CHAMPIONNAME",'TEAMCHAMPION']).sum().reset_index()
champ_combination["RATE"] = round(champ_combination.WIN / champ_combination.games * 100 ,1)
combination_rate = pd.merge(win_rate,champ_combination, on=["CHAMPIONNAME"]).drop(['WIN_x','WIN_y'],axis=1)\
                                                        .rename(columns={"rate":"default_rate","RATE":"combination_rate"})
combination_rate["COMBINATION_SCORE"] = combination_rate.combination_rate - combination_rate.default_rate

combination_rate["rank"] = combination_rate.apply(lambda x : rank(x), axis=1)
#     combination_rate[combination_rate.TEAMCHAMPION =="FiddleSticks"] = combination_rate[combination_rate.TEAMCHAMPION 
#                                                                                         =="FiddleSticks"].replace("FiddleSticks","Fiddlesticks")
combination_rate["SYNERGY_TIER"] = combination_rate.apply(lambda x: x.TEAMCHAMPION, axis=1)
combination_rate = combination_rate[["CHAMPIONNAME","TEAMCHAMPION","COMBINATION_SCORE","rank","SYNERGY_TIER"]]

champ_lst = pd.read_csv("CHAMP_LST.csv")
champ_lst = champ_lst[["CHAMPIONNAME","CHAMPNAME"]]
champ_lst = champ_lst.rename(columns={"CHAMPIONNAME":"SYNERGY_TIER"})

cr2 = pd.merge(combination_rate, champ_lst, on="SYNERGY_TIER").sort_values("COMBINATION_SCORE", ascending=False)
cr2["COMBINATION_SCORE"] = cr2.apply(lambda x: score_str(x), axis=1)

# combination_rate.sort_values("COMBINATION_SCORE",ascending=False, inplace=True)

# combination_rate["COMBINATION_SCORE"] = combination_rate.apply(lambda x: score_str(x), axis=1)


combination_rate_img = cr2.groupby(["CHAMPIONNAME","rank"])["SYNERGY_TIER"].apply(lambda x: "%s" % '|'.join(x))
combination_rate_combination_score = cr2.groupby(["CHAMPIONNAME","rank"])["COMBINATION_SCORE"].apply(lambda x: "%s" % '|'.join(x))
combination_rate_ko_name = cr2.groupby(["CHAMPIONNAME","rank"])["CHAMPNAME"].apply(lambda x: "%s" % '|'.join(x))


combination_rate_img = pd.DataFrame(combination_rate_img)
combination_rate_img.reset_index(inplace=True)
combination_rate_img = combination_rate_img.groupby(["CHAMPIONNAME"])["SYNERGY_TIER"].apply(lambda x: "%s" % ','.join(x))

combination_rate_combination_score = pd.DataFrame(combination_rate_combination_score)
combination_rate_combination_score.reset_index(inplace=True)
combination_rate_combination_score = combination_rate_combination_score.groupby(["CHAMPIONNAME"])["COMBINATION_SCORE"].apply(lambda x: "%s" % 
                                                                                                                             ','.join(x))

combination_rate_ko_name = pd.DataFrame(combination_rate_ko_name)
combination_rate_ko_name.reset_index(inplace=True)
combination_rate_ko_name = combination_rate_ko_name.groupby(["CHAMPIONNAME"])["CHAMPNAME"].apply(lambda x: "%s" % ','.join(x))


combination_rate_img = pd.DataFrame(combination_rate_img)
combination_rate_combination_score = pd.DataFrame(combination_rate_combination_score)
combination_rate_ko_name = pd.DataFrame(combination_rate_ko_name)

synergy1 = pd.merge(combination_rate_img,combination_rate_combination_score, on=["CHAMPIONNAME"]).reset_index()
synergy = pd.merge(synergy1, combination_rate_ko_name ,on="CHAMPIONNAME")

In [42]:
combination_rate_ko_name.loc["Aatrox"]["CHAMPNAME"].split(",")[0].split("|")

['사이온',
 '나미',
 '소나',
 '자야',
 '올라프',
 '블라디미르',
 '아이번',
 '코그모',
 '세라핀',
 '볼리베어',
 '갈리오',
 '유미',
 '잔나',
 '룰루',
 '자이라',
 '마오카이',
 '애쉬',
 '카타리나',
 '빅토르',
 '레오나',
 '애니비아',
 '오리아나',
 '리산드라',
 '베이가',
 '그레이브즈',
 '직스',
 '모르가나',
 '알리스타',
 '바루스']

In [113]:
synergy1 = pd.merge(synergy, combination_rate_ko_name ,on="CHAMPIONNAME").rename(columns={"CHAMPNAME":"K_SYNERGY"})

In [124]:
synergy.iloc[137]["SYNERGY_TIER"].split(",")[0].split("|")

['Ivern',
 'Kled',
 'Gnar',
 'Nasus',
 'Nami',
 'Senna',
 'Lulu',
 'Caitlyn',
 'Gwen',
 'RekSai',
 'Sona',
 'Graves',
 'Ashe',
 'Sett',
 'TahmKench',
 'Varus',
 'Rakan',
 'Riven',
 'Hecarim',
 'Qiyana',
 'Tryndamere',
 'Sejuani',
 'Yuumi',
 'Samira',
 'Lillia',
 'Jinx',
 'Jayce',
 'Draven',
 'XinZhao']

In [125]:
synergy.iloc[137]["CHAMPNAME"].split(",")[0].split("|")

['아이번',
 '클레드',
 '나르',
 '나서스',
 '나미',
 '세나',
 '룰루',
 '케이틀린',
 '그웬',
 '렉사이',
 '소나',
 '그레이브즈',
 '애쉬',
 '세트',
 '탐 켄치',
 '바루스',
 '라칸',
 '리븐',
 '헤카림',
 '키아나',
 '트린다미어',
 '세주아니',
 '유미',
 '사미라',
 '릴리아',
 '징크스',
 '제이스',
 '드레이븐',
 '신 짜오']

In [9]:
synergy = combination(df)

In [9]:
champ_lst = pd.read_csv("CHAMP_LST.csv")
champ_lst = champ_lst[["CHAMPIONNAME","CHAMPNAME"]]
champ_lst = champ_lst.rename(columns={"CHAMPIONNAME":"SYNERGY_TIER"})
combination_rate = pd.merge(combination_rate, champ_lst, on="SYNERGY_TIER")

In [29]:
combination_rate_ko_name = combination_rate.groupby(["CHAMPIONNAME","rank"])["CHAMPNAME"].apply(lambda x: "%s" % '|'.join(x))

## -----------------------------------------------------------------------------------------------------------------------------

In [6]:
def rank(row):
    if 20 > row.COMBINATION_SCORE >= 5:
        return "A"
    elif 5 > row.COMBINATION_SCORE > -5:
        return "C"
    elif -30 < row.COMBINATION_SCORE < -5:
        return "B"

In [7]:
def score_str(row):
    a = "{:.1f}%".format(row.COMBINATION_SCORE)
    return a

In [373]:
# 최종적으로 스프링단에 표현할때 아래와 같이 써주면 될 것 같다.
A = testing3.iloc[0]["img"].split(",")[0].split("|")
A_RATE = testing3.iloc[0]["combination_score"].split(",")[0].split("|")
B = testing3.iloc[0]["img"].split(",")[1].split("|")
B_RATE = testing3.iloc[0]["combination_score"].split(",")[1].split("|")